In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable

import pandas as pd
import random
import string
import numpy as np

import sys, os

import torch.utils.data as data

os.environ["CUDA_VISIBLE_DEVICES"] = '0'


all_characters = string.printable
number_of_characters = len(all_characters)

artists = [
'ABBA',
'Ace Of Base',
'Aerosmith',
'Avril Lavigne',
'Backstreet Boys',
'Bob Marley',
'Bon Jovi',
'Britney Spears',
'Bruno Mars',
'Coldplay',
'Def Leppard',
'Depeche Mode',
'Ed Sheeran',
'Elton John',
'Elvis Presley',
'Eminem',
'Enrique Iglesias',
'Evanescence',
'Fall Out Boy',
'Foo Fighters',
'Green Day',
 'HIM',
 'Imagine Dragons',
 'Incubus',
 'Jimi Hendrix',
 'Justin Bieber',
 'Justin Timberlake',
'Kanye West',
 'Katy Perry',
 'The Killers',
 'Kiss',
 'Lady Gaga',
 'Lana Del Rey',
 'Linkin Park',
 'Madonna',
 'Marilyn Manson',
 'Maroon 5',
 'Metallica',
 'Michael Bolton',
 'Michael Jackson',
 'Miley Cyrus',
 'Nickelback',
 'Nightwish',
 'Nirvana',
 'Oasis',
 'Offspring',
 'One Direction',
 'Ozzy Osbourne',
 'P!nk',
 'Queen',
 'Radiohead',
 'Red Hot Chili Peppers',
 'Rihanna',
 'Robbie Williams',
 'Rolling Stones',
 'Roxette',
 'Scorpions',
 'Snoop Dogg',
 'Sting',
 'The Script',
 'U2',
 'Weezer',
 'Yellowcard',
 'ZZ Top']


def character_to_label(character):
    """Returns a one-hot-encoded tensor given a character.
    
    Uses string.printable as a dictionary.
        
    Parameters
    ----------
    character : str
        A character
        
    Returns
    -------
    one_hot_tensor : Tensor of shape (1, number_of_characters)
        One-hot-encoded tensor
    """
    
    character_label = all_characters.find(character)
        
    return character_label


def string_to_labels(character_string):
    
    return map(lambda character: character_to_label(character), character_string)


def pad_sequence(seq, max_length, pad_label=100):
    
    seq += [pad_label for i in range(max_length - len(seq))]
    
    return seq


class LyricsGenerationDataset(data.Dataset):
    
    def __init__(self, csv_file_path, minimum_song_count=None, artists=None):
        
        
        self.lyrics_dataframe = pd.read_csv(csv_file_path)
        
        if artists:
            
            self.lyrics_dataframe = self.lyrics_dataframe[self.lyrics_dataframe.artist.isin(artists)]
            self.lyrics_dataframe = self.lyrics_dataframe.reset_index()
        
        if minimum_song_count:
        
            # Getting artists that have 70+ songs
            self.lyrics_dataframe = self.lyrics_dataframe.groupby('artist').filter(lambda x: len(x) > minimum_song_count)
            # Reindex .loc after we fetched random songs
            self.lyrics_dataframe = self.lyrics_dataframe.reset_index()
        
        # Get the length of the biggest lyric text
        # We will need that for padding
        self.max_text_len = self.lyrics_dataframe.text.str.len().max()
        
        whole_dataset_len = len(self.lyrics_dataframe)
        
        self.indexes = range(whole_dataset_len)
        
        
        # Let's get unique artists and form a list
        self.artists_list = list(self.lyrics_dataframe.artist.unique())
        
        # We will need the overall number of artists for 
        self.number_of_artists = len(self.artists_list)
    
    
    def __len__(self):
        
        return len(self.indexes)
    
    
    def __getitem__(self, index):
        
        index = self.indexes[index]
        
        sequence_raw_string = self.lyrics_dataframe.loc[index].text
        
        sequence_string_labels = string_to_labels(sequence_raw_string)
        
        sequence_length = len(sequence_string_labels) - 1
        
        # Shifted by one char
        input_string_labels = sequence_string_labels[:-1]
        output_string_labels = sequence_string_labels[1:]
                
        # pad sequence so that all of them have the same lenght
        # Otherwise the batching won't work
        input_string_labels_padded = pad_sequence(input_string_labels, max_length=self.max_text_len)
        
        output_string_labels_padded = pad_sequence(output_string_labels, max_length=self.max_text_len, pad_label=-100)
        
        ## Adding the artist label
        sequence_raw_artist_name_string = self.lyrics_dataframe.loc[index].artist

        sequence_artist_label = self.artists_list.index(sequence_raw_artist_name_string)
        
        return (torch.LongTensor(input_string_labels_padded),
                torch.LongTensor(output_string_labels_padded),
                torch.LongTensor([sequence_artist_label]),
                torch.LongTensor([sequence_length]) )

    
def post_process_sequence_batch(batch_tuple):
    
    input_sequences, output_sequences, artists, lengths = batch_tuple
    
    splitted_input_sequence_batch = input_sequences.split(split_size=1)
    splitted_output_sequence_batch = output_sequences.split(split_size=1)
    splitted_artists_batch = artists.split(split_size=1)
    splitted_lengths_batch = lengths.split(split_size=1)

    training_data_tuples = zip(splitted_input_sequence_batch,
                               splitted_output_sequence_batch,
                               splitted_artists_batch,
                               splitted_lengths_batch)

    training_data_tuples_sorted = sorted(training_data_tuples,
                                         key=lambda p: int(p[3]),
                                         reverse=True)

    splitted_input_sequence_batch, splitted_output_sequence_batch, splitted_artists_batch, splitted_lengths_batch = zip(*training_data_tuples_sorted)

    input_sequence_batch_sorted = torch.cat(splitted_input_sequence_batch)
    output_sequence_batch_sorted = torch.cat(splitted_output_sequence_batch)
    artists_batch_sorted = torch.cat(splitted_artists_batch)
    lengths_batch_sorted = torch.cat(splitted_lengths_batch)
    
    
    # Here we trim overall data matrix using the size of the longest sequence
    input_sequence_batch_sorted = input_sequence_batch_sorted[:, :lengths_batch_sorted[0, 0]]
    output_sequence_batch_sorted = output_sequence_batch_sorted[:, :lengths_batch_sorted[0, 0]]
    
    # We should probably repeat this over the whole input sequence
    artists_batch_sorted = artists_batch_sorted.expand_as(input_sequence_batch_sorted)

    input_sequence_batch_transposed = input_sequence_batch_sorted.transpose(0, 1)
    artists_batch_sorted_transposed = artists_batch_sorted.transpose(0, 1)
    
    # pytorch's api for rnns wants lenghts to be list of ints
    lengths_batch_sorted_list = list(lengths_batch_sorted)
    lengths_batch_sorted_list = map(lambda x: int(x), lengths_batch_sorted_list)
    
    return input_sequence_batch_transposed, output_sequence_batch_sorted, artists_batch_sorted_transposed, lengths_batch_sorted_list


class RNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_classes, num_conditions, n_layers=2):
        
        super(RNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_classes = num_classes
        self.n_layers = n_layers
        self.num_conditions = num_conditions
        
        # Converts labels into one-hot encoding and runs a linear
        # layer on each of the converted one-hot encoded elements
        
        # input_size -- size of the dictionary + 1 (accounts for padding constant)
        self.characters_encoder = nn.Embedding(input_size, hidden_size)
        
        self.conditions_encoder = nn.Embedding(num_conditions, hidden_size)
        
        self.lstm = nn.LSTM(hidden_size * 2, hidden_size, n_layers)
        
        self.logits_fc = nn.Linear(hidden_size, num_classes)
    
    
    def forward(self, input_sequences, input_sequences_conditions, input_sequences_lengths, hidden=None):
        
        batch_size = input_sequences.shape[1]

        characters_encoded = self.characters_encoder(input_sequences)
        conditions_endoded = self.conditions_encoder(input_sequences_conditions)
        
        encodings_combined = torch.cat((characters_encoded, conditions_endoded), dim=2)

        # Here we run rnns only on non-padded regions of the batch
        packed = torch.nn.utils.rnn.pack_padded_sequence(encodings_combined, input_sequences_lengths)
        outputs, hidden = self.lstm(packed, hidden)
        outputs, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(outputs) # unpack (back to padded)
        
        logits = self.logits_fc(outputs)
        
        logits = logits.transpose(0, 1).contiguous()
        
        logits_flatten = logits.view(-1, self.num_classes)
        
        return logits_flatten, hidden


trainset = LyricsGenerationDataset(csv_file_path='songdata.csv', artists=artists)

trainset_loader = torch.utils.data.DataLoader(trainset, batch_size=50,
                                              shuffle=True, num_workers=4, drop_last=True)


In [2]:
rnn = RNN(input_size=len(all_characters) + 1,
          hidden_size=512,
          num_classes=len(all_characters),
          num_conditions=trainset.number_of_artists)

rnn.cuda()

learning_rate = 0.001
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss().cuda()

def sample_from_rnn_conditionally(starting_sting="Why", sample_length=300, temperature=1, artist_label=0):
    
    sampled_string = starting_sting
    hidden = None

    first_input = torch.LongTensor( string_to_labels(starting_sting) ).cuda()
    first_input = first_input.unsqueeze(1)

    # Expand the artist label to have the same size as input sequence
    # we duplicate it in every input
    artist_label_input = torch.LongTensor([artist_label]).expand_as(first_input)

    current_sequence_input = Variable(first_input)
    current_artist_input = Variable(artist_label_input.cuda())

    output, hidden = rnn(current_sequence_input, current_artist_input, [len(sampled_string)], hidden=hidden)

    output = output[-1, :].unsqueeze(0)

    for i in xrange(sample_length):

        output_dist = nn.functional.softmax( output.view(-1).div(temperature) ).data

        predicted_label = torch.multinomial(output_dist, 1)

        sampled_string += all_characters[int(predicted_label[0])]
        current_sequence_input = Variable(predicted_label.unsqueeze(1))

        artist_label_input = torch.LongTensor([artist_label]).expand_as(current_sequence_input)
        current_artist_input = Variable(artist_label_input.cuda())

        output, hidden = rnn(current_sequence_input, current_artist_input, [1], hidden=hidden)
    
    return sampled_string

In [4]:
%matplotlib notebook

from matplotlib import pyplot as plt

# importing a list that can be binded with a figure and update
# the figure when being appended
from visualization import VizList

# Creating figure, axes and binding to lists 
f, (loss_axis, validation_axis, train_axis) = plt.subplots(3, 1)

loss_axis.plot([], [])
validation_axis.plot([], [])
train_axis.plot([], [])


loss_list = VizList()
validation_list = VizList()
train_list = VizList()

loss_list.bind_to_axis(loss_axis)
validation_list.bind_to_axis(validation_axis)
train_list.bind_to_axis(train_axis)

loss_axis.set_title('Training Loss')
validation_axis.set_title('Validation Set Accuracy')
train_axis.set_title('Training Set Accuracy')


plt.tight_layout()

<IPython.core.display.Javascript object>

In [5]:
clip = 1.0
epochs_number = 1000

for epoch_number in xrange(epochs_number):

    for batch in trainset_loader:

        post_processed_batch_tuple = post_process_sequence_batch(batch)

        input_sequences_batch, output_sequences_batch, artists_batch, sequences_lengths = post_processed_batch_tuple

        output_sequences_batch_var =  Variable( output_sequences_batch.contiguous().view(-1).cuda() )
        
        
        input_sequences_batch_var = Variable( input_sequences_batch.cuda() )
        artists_batch_var = Variable( artists_batch.cuda() )
        
        optimizer.zero_grad()

        logits, _ = rnn(input_sequences_batch_var, artists_batch_var, sequences_lengths)
        
        loss = criterion(logits, output_sequences_batch_var)
        loss_list.append( loss.data[0] )
        loss.backward()

        #torch.nn.utils.clip_grad_norm(rnn.parameters(), clip)

        optimizer.step()
        
    print(sample_from_rnn_conditionally(artist_label=trainset.artists_list.index("Kanye West")))
    torch.save(rnn.state_dict(), 'conditional_rnn.pth')
    

/home/daniil/repos/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Why  
Girl sure ray sromen braty cooder anbona  
I staC  
Yeantighta hing, f-raped stans to man't stome onsile bacr,  
Leah me lamen  
My lanked tall if stuad it to tom no zeack on the boytle to munshide think of for my comestess it so caah feeferno me  
Even dreakin' havin' lan  
(osiil you wan telcer
Why shake I mon it wicker me  
Don't never long, oh!, I don't zeomin' thannit more you, Leae, a futous, grun  
I'm min't just still the hand sceet me in...  
 -Bup-as-ge-[Chis, lood fay  
Being line Gons  
We wastedmy something estay kind) without in this of you, hand  
Cause of then down  
  
It's rig
Why nobody will stand Kyess  
And you Calling if you are the still the parpho fun  
  
In the tilk grists my neyte  
You guick on the could bentyry now  
Put thought Ted Dodry  
Are you get a kill outterne.  
Hand embel up  
Onow in friends thross chitting  
Don't phtent at I'll be might?  
If Benaches
Whyme, talkin' for you like What's I don't want give me exact  
Undight, Every-No]

Now 

Why don't you counting the page?  
It seems I'd rather stand up? I was all you can't find sayin  
And if you pretend need to be in the morning  
And I can't disman in the colden shirt.  
No uh huh, that's a remainsteblic 'smas nigga  
I know back the key to you,  
This is revenge of whoa  
No matter ho
Why don't you know?  
The way hey  
What am I doin'?  
When I could only do  
  
[Verse 2: Michaella]  
And go wrong I've got to rich my only  
Can't spicitude like and I could ever do  
I thought I'll admit  
  
[Verse 2]  
Yeah I'm gone going light? And I'm livin' in  
If I just Kleet like freezin' (
Why can't you see?  
We had a coop for lights  
And there's nothin' ever worry obsessed kind  
Another big bob you took you down  
Just let the duck out speed  
We can move a while or to lie (feels busted) 
  
Verse:  
Keep your chance  
You gave your bag you will not  
You came for me  
We getting you
Why don meet y'all?  
They bother motherfucking luck or was niggers got right?  
Robber 

Why you obey? You really live?  
I just want to fly  
I told you I'm not bringing a girl like you

So it's now or not  
  
You see G. O. S.O.O.b. But  
And your stones deep then livin' shit  
So far away from a horrok ass dead  
You don't hide it out there masterboard, so the wrong poor  
The Repty Fuc
Why'cha' you crawl  
  
[CHoop:]  
No, no, no, no no, no, no, no  
What that he do is  
We're all around from me  
  
[Verse: Madelin]  
So  
I've never been  
Not all they getted from  
When loom to Drink at palm  
How could they call [Va? x 2, for it Blues, that's Rhandra's daydream  
You ain't take 
Why'am  
I thought we had it for life  
I tried to make you feel like I had  
To numberbance in land  
In this life we open a place perfect I know

You got it opported on  
But if you only hate hurts  
But when I see somethin' else bring  
Baby  
When I show 'em all  
  
And the girls talk  
That can s
Why I know what I'm thinkin?  
Just like then be on my big knees  
Do you think she give

Why can't we fly, all I ever told you bring yo dem a dem a difference  
If everybody else will be our glory is to blame  
They take you to the general of the yaching  
They ain't speakin' from you but they ain't  
I love you, all the worst is over  
  
I gave a damn tooth, the day that I receive it  
I
Why are you cool if your lone?  
(Wis one time to come to this?  
I'd like to appear when night?  
(I hate this ole with you, with no shot, from the side?)  
Everything in that homie go go  
Girl till my eyes come to stay solvine  
Goodbye my skin years off for all my grind, yeah  
Give you no flashing
Why would you be awesed for a forming strew  
How many younger played tomorrow consumination?  
Emily duck to the beats forever true  
My mama say on he wear I'm so rolled out since Maryly  
Now I don't need welt you graduation to the likem,  
A song called the of the girls, that they keep it?  
I'm Mi
Why (Yo, y)  
IGlone, baby  
Unless what you want  
Is what I paint some lonely  
Turn t

...

Why you're sure to do?  
Why'd about you about the stars?  
Who ain't know it ain't everything,  
that's in this bumpin'  
I know I'm true  
If it wasn't mine, and he check off  
  
Somebody pay a nigga jumpin' in  
And alcohole in Grac  
Shorties full of city  
With no somethin' like you all five  
  
Why she talkin' 'bout my damn way, yeah  
Till he come on making it she meant nothing she like  
She in, with a nigga guns  
He was everything he can do it hurt  
But he seen his old chick and black and five  
I used to listen to the ass-sepper, fareota-keekee  
He couched her technichesel I needed to 
Why everybody scream.  
But so amazin', she can smitch it off  
When she hose like I wouldn't kill him perferin',  
While 'till she'll chose embrask"  
Mixy Eself was severe (so he gonna holle)  
If she looked like you about being nothin', no  
Like Jesus Wanna live in Nem and Jaw,  
Now that he could 
Why would she don't stop?  
When they can feel it?s can't put them pleaded out of my fav

Why are you sincerator, your gold and yellow?  
You from away out and you  
All the pachaa then  
Who cursers start to rollin' as a rest  
To legs you got the eyes against a hard enough  
And to get through to young must be more gone joine  
You hate all you erased  
And I'd like to tell you no commoti
Why are you sick-Duse the one I got? (Ain), dying fool,  
Leaving your harm back in line! OUTah!  
Letting out your firencluses  
L-- there's some me, so let me may be that'll rely  
Every time I'm sad  
  
How many page, huh, huh?  
But that's blowin' shine bright (Trick)  
Dreaming of my steps in the
Why are you supposed to be the same thing  
So you talkin' to the morning shift we'd like to be honestly  
See you havin' nowhere cryin 'cause I'm in love wit' it  
I'm the only one that I know is almostalls  
  
And I always find way  
Till only I found because I know  
We bag up in the club and key a
Why are you so crazy...Pintic  
So if you can you hear the later in the end  
What the f

Why are you supposed to be?  
Told that he got something we get this feeling?  
You eyes with a few full of gold  
I hear no more straight to hell, this celebration  
Way our heart acts already cries  
  
Memorization (Liar like when you drop that real life)  
With you on the streets of our was X2  
I 
Why are you saved your memory?meless?  
Can't wait for the laddest?  
Don't want your nossy, no you're from  
Let's get me so here right now  
In the skies at the invrility now  
I checking every worst just you and pull  
Just an ass gonna die, never fear  
I know I take a number hade  
  
Now we outta
Why are you so young now 'till you'll be fillin' the door  
Still, forever this, so just take the shit  
You waitin' for the way she get high  
A beautiful a head command all the time  
  
[Verse 1: Hook]  
Anyone  
Man you know I ain't goin go on  
When it don't feel so good  
On the moon when I'm dow
Why are you so paranoid?  
Can anything on the leval mention and  
Youve got a sadness d

Why are you so proud of man  
Games Do our April, Benevished  
Divinna like a flock  
Just because  
As it sucks gon be on  
A coup-remedimes rising  
'Cause life here went were old  
Been stoppin' by the Emends  
Life is switch and say  
Like white flash-out  
Cus [It's Dort and second Gia)  
Fuck ya 
Why as love was used to love you?  
Wishes forgotten lighted by my lips just to hear you breathe in your life, yeah if I loved you for you no more  
Singing so though? I wouldn't live you everything nigga  
I'd do it for a webritch, you treat me hypn  
  
And my friend and may be good  
My reason for m
Why not do you put it down yeard?
?Dreck it?  
I'm not like the fast stority  
They don't see everything  
  
And you see me in your eyes  
And sleep the door, pop and leters  
And I'm caught up in the conscious  
And I'm gone, I'm so green just a lighter  
I'd like to suffer, why should I creep in it 
Why as love could I?  
I might be more than left me  
Then I spill hang a flag  
I may b

Why would stick like a sea?  
Who might think it's about me that shit? Won't  
Why am I gonna lead you on my feeling

One shit that's so greedy  
  
Euph of many things so then we're asking ours to get a candle break  
The hatin' life was a world that I know did what I did  
All the little whalling of 
